In [1]:
# Quantum
import pennylane as qml
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Numpy, Pandas
import numpy as np
import pandas as pd
# Layer
from kan import KAN
from RNN_block import RNN_block
# Data processing
from fucntions import data_seq, train_seq
from sklearn.preprocessing import MinMaxScaler

### Settings

In [2]:
n_qu = 5
dev = qml.device('default.qubit', wires = n_qu)

In [3]:
def embedding(params, n_qu):
    '''
    embedding layer
    '''
    n = n_qu
    for i in range(n):
        qml.Hadamard(i)
        qml.RZ(2.0 * params[ : , i], i)
     
    for i in range(n - 1):
        qml.IsingZZ(2.0 * params[ : , n + i] , [i, i + 1])

@qml.qnode(dev, interface = "torch")
def fidelity(vec1, vec2, n_qu):
    '''
        Args:
            vec1 : list, (2n - 1)개의 element로 이루어진 vector
            vec2 : list, (2n - 1)개의 element로 이루어진 vector
    '''
    embedding(vec1, n_qu) # Phi(x1) circuit 적용
    qml.adjoint(embedding)(vec2, n_qu) # Phi^t(x2) 적용
    return qml.probs()

## NQE Class

In [15]:
class NQE(nn.Module):
    def __init__(self, n_feature, mode : str):
        '''
            Args:
                type(str) : 'FC' or 'KAN'
                n_feature(int) : # of feature
        '''
        super(NQE, self).__init__()

        self.mode = mode

        if mode == 'FC':
            self.li1 = nn.Linear(n_feature, n_feature * n_feature)
            self.li2 = nn.Linear(n_feature * n_feature, n_feature * n_feature)
            self.li3 = nn.Linear(n_feature * n_feature, 2 * n_feature - 1)
        
        if mode == 'KAN':
            self.n_qu = n_feature
            self.linear1 = KAN([self.n_qu, self.n_qu * 2 + 1, self.n_qu * 2 - 1], grid = 1)
            self.quantum_layer = fidelity

    def forward_FC(self, inputs):
        inputs = self.li1(inputs)
        inputs = F.relu(inputs)
        inputs = self.li2(inputs)
        inputs = F.relu(inputs)
        inputs = self.li3(inputs)
        result = 2 * torch.pi * F.relu(inputs)
        ## Quantum Layer 추가 필요
        return result # Quantum Layer의 output을 리턴

    def forward_KAN(self, inputs):
        input1 = inputs[0]
        input2 = inputs[1]
        input1 = self.linear1(input1)
        input2 = self.linear1(input2)
        output = self.quantum_layer(input1, input2, self.n_qu)[ : , 0]
        return output

    def forward(self, inputs):
        if self.mode == 'FC':
            return self.forward_FC(inputs)
        if self.mode == 'KAN':
            return self.forward_KAN(inputs)

## Data uploading

In [16]:
locations = ['Adelaide', 'Albany', 'Albury', 'AliceSprings', 'BadgerysCreek', 'Ballarat', 'Bendigo', 'Brisbane', 'Cairns', 'Canberra', 'Cobar', 'CoffsHarbour', 'Dartmoor', 'Darwin', 'GoldCoast', 'Hobart', 'Katherine', 'Launceston', 'Melbourne', 'MelbourneAirport', 'Mildura', 'Moree', 'MountGambier', 'MountGinini', 'Newcastle', 'Nhil', 'NorahHead', 'NorfolkIsland', 'Nuriootpa', 'PearceRAAF', 'Penrith', 'Perth', 'PerthAirport', 'Portland', 'Richmond', 'Sale', 'SalmonGums', 'Sydney', 'SydneyAirport', 'Townsville', 'Tuggeranong', 'Uluru', 'WaggaWagga', 'Walpole', 'Watsonia', 'Williamtown', 'Witchcliffe', 'Wollongong', 'Woomera']

In [17]:
nqe_train = 200

In [18]:
train_data_dict = dict()
label_data_dict = dict()
for e in locations:
    print(e)
    train_df = pd.read_csv("./data/train_data_" + e + ".csv")
    label_df = pd.read_csv("./data/label_data_" + e + ".csv")
    train_data_dict[e] = torch.tensor(train_df[["MinTemp","MaxTemp","Rainfall","Humidity3pm","Pressure3pm"]].to_numpy()[:nqe_train]).to(torch.float)
    label_data_dict[e] = torch.tensor(label_df['RainTomorrow'].to_numpy()[:nqe_train]).to(torch.float)


Adelaide
Albany
Albury
AliceSprings
BadgerysCreek
Ballarat
Bendigo
Brisbane
Cairns
Canberra
Cobar
CoffsHarbour
Dartmoor
Darwin
GoldCoast
Hobart
Katherine
Launceston
Melbourne
MelbourneAirport
Mildura
Moree
MountGambier
MountGinini
Newcastle
Nhil
NorahHead
NorfolkIsland
Nuriootpa
PearceRAAF
Penrith
Perth
PerthAirport
Portland
Richmond
Sale
SalmonGums
Sydney
SydneyAirport
Townsville
Tuggeranong
Uluru
WaggaWagga
Walpole
Watsonia
Williamtown
Witchcliffe
Wollongong
Woomera


In [19]:
nqe_x_train = train_data_dict[locations[0]]
nqe_y_train = label_data_dict[locations[0]]

In [20]:
nqe_train_list = []
nqe_train_label_list = []
nqe_test_list = []
nqe_test_label_list = []

for i in range(nqe_train-1):
    nqe_train_data = torch.stack([nqe_x_train, torch.concat([nqe_x_train[(i + 1) : ], nqe_x_train[ : (i + 1)]])])
    nqe_train_list.append(nqe_train_data)
    nqe_label_data = torch.stack([nqe_y_train,torch.concat([nqe_y_train[(i + 1) :],nqe_y_train[: (i + 1)]])])
    nqe_train_label_list.append(nqe_label_data)

In [21]:
nqe_train_data = torch.concat(nqe_train_list, dim = 1)
nqe_train_label = torch.concat(nqe_train_label_list, dim = 1)

In [22]:
print(nqe_train_data.shape)
print(nqe_train_label.shape)

torch.Size([2, 39800, 5])
torch.Size([2, 39800])


## NQE Train

In [23]:
class NQE_Train:
    def __init__(self, nqe, criterion, data_pretrain):
        '''
            Args:
                nqe (NQE) : nqe object want to train
                criterion (function) : loss function
                data_pretrain (data_seq) : want to make train_seq
                optimizer (torch.optimizer)
        '''
        self.nqe = nqe
        self.loss = criterion
        self.pretrain_data = data_pretrain
        self.optim = optim.Adam(self.nqe.parameters(), lr = 0.005)
    
    def train(self, epoch):
        nqe_train_loader, nqe_test_loader = self.pretrain_data.split_data(batch_size = 64, seq_first = True)
        nqe_seq = train_seq(self.nqe, nqe_train_loader, nqe_test_loader)
        nqe_seq.train(epoch, self.optim, self.loss, seq_first=True)
        return self.nqe

In [24]:
def criterion(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    # print(pred.shape)
    # print(label.shape)
    return torch.sum(((pred ** 2) - 0.5 * (label[:, 0] * label[:, 1] + 1)) ** 2 )/len(pred)

In [25]:
n_qu = 5 # number of features
nqe1 = NQE(n_qu, 'KAN')
nqe_train1 = NQE_Train(nqe1, criterion, data_seq(nqe_train_data, nqe_train_label))

In [26]:
trained_nqe = nqe_train1.train(10)

epoch : 1 loss :0.586495041847229 loss_test = 0.5870387554168701
epoch : 2 loss :0.5866676568984985 loss_test = 0.587451696395874
epoch : 3 loss :0.5865719318389893 loss_test = 0.5876114368438721
epoch : 4 loss :0.5866602063179016 loss_test = 0.5874125957489014
epoch : 5 loss :0.5867141485214233 loss_test = 0.5875712633132935
epoch : 6 loss :0.5867512822151184 loss_test = 0.5869946479797363
epoch : 7 loss :0.5865515470504761 loss_test = 0.5872092247009277
epoch : 8 loss :0.5866084694862366 loss_test = 0.5874603986740112
epoch : 9 loss :0.5867410898208618 loss_test = 0.5874874591827393
epoch : 10 loss :0.5866948962211609 loss_test = 0.5873752236366272
